In [ ]:
# ✅ 1. Install required libraries
!pip install -q torch torchvision torchaudio pandas scikit-learn nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# ✅ 3. Imports
import os
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

In [ ]:
# ✅ 4. Hyperparameters and Save Path
MAX_LEN = 200
BATCH_SIZE = 32
EMBEDDING_DIM = 128
HIDDEN_DIM = 128
NUM_CLASSES = 2
EPOCHS = 5
LR = 1e-3
SAVE_DIR = "/content/drive/MyDrive/MultiBanFake/LSTM_Results"
os.makedirs(SAVE_DIR, exist_ok=True)

In [ ]:
# ✅ 5. Load and Prepare Data
train_df = pd.read_csv("/content/drive/MyDrive/MultiBanFake/Dataset/text/Train.csv")
val_df = pd.read_csv("/content/drive/MyDrive/MultiBanFake/Dataset/text/Validation.csv")
test_df = pd.read_csv("/content/drive/MyDrive/MultiBanFake/Dataset/text/Test.csv")

for df in [train_df, val_df, test_df]:
    df["headline"] = df["headline"].fillna("").astype(str)
    df["description"] = df["description"].fillna("").astype(str)
    df["text"] = df["headline"].str.strip() + " " + df["description"].str.strip()


In [ ]:
# ✅ 6. Build Vocabulary
def simple_tokenize(text):
    return re.findall(r"\b\w+\b", text.lower())

from collections import Counter
all_texts = train_df["text"].tolist()
tokenized_texts = [simple_tokenize(t) for t in all_texts]
word_counts = Counter(token for sent in tokenized_texts for token in sent)

vocab = {"<PAD>": 0, "<UNK>": 1}
for word, freq in word_counts.items():
    if freq >= 2:
        vocab[word] = len(vocab)

In [ ]:
# ✅ 7. Encode text
def encode(text, max_len=MAX_LEN):
    tokens = simple_tokenize(text)
    ids = [vocab.get(token, vocab["<UNK>"]) for token in tokens]
    if len(ids) < max_len:
        ids += [vocab["<PAD>"]] * (max_len - len(ids))
    else:
        ids = ids[:max_len]
    return torch.tensor(ids)

In [ ]:
# ✅ 8. Custom Dataset
class NewsDataset(Dataset):
    def __init__(self, df):
        self.texts = df["text"].tolist()
        self.labels = df["label"].astype(int).tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {
            "input": encode(self.texts[idx]),
            "label": torch.tensor(self.labels[idx], dtype=torch.long)
        }

train_dataset = NewsDataset(train_df)
val_dataset = NewsDataset(val_df)
test_dataset = NewsDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)


In [ ]:
# ✅ 9. LSTM Model
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        _, (hn, _) = self.lstm(x)
        out = self.fc(hn[-1])
        return out

model = LSTMClassifier(len(vocab), EMBEDDING_DIM, HIDDEN_DIM, NUM_CLASSES)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


LSTMClassifier(
  (embedding): Embedding(5174, 128, padding_idx=0)
  (lstm): LSTM(128, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=2, bias=True)
)

In [ ]:
# ✅ 10. Train Loop
optimizer = optim.Adam(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for batch in train_loader:
        inputs = batch["input"].to(device)
        labels = batch["label"].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"✅ Epoch {epoch+1}/{EPOCHS} - Loss: {total_loss:.4f}")


✅ Epoch 1/5 - Loss: 167.1129
✅ Epoch 2/5 - Loss: 164.9368
✅ Epoch 3/5 - Loss: 160.6281
✅ Epoch 4/5 - Loss: 147.8611
✅ Epoch 5/5 - Loss: 124.1390


In [ ]:
# ✅ 11. Save Model
model_path = os.path.join(SAVE_DIR, "lstm_model.pt")
torch.save(model.state_dict(), model_path)
print(f"✅ Model saved to: {model_path}")

# ✅ 12. Evaluation
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        inputs = batch["input"].to(device)
        labels = batch["label"].to(device)
        outputs = model(inputs)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


✅ Model saved to: /content/drive/MyDrive/MultiBanFake/LSTM_Results/lstm_model.pt


In [ ]:
# ✅ 13. Save Results
from datetime import datetime

acc = accuracy_score(all_labels, all_preds)
report = classification_report(all_labels, all_preds, target_names=["Non-Fake", "Fake"])

# Save predictions
results_df = pd.DataFrame({
    "true_label": all_labels,
    "predicted_label": all_preds
})
results_csv = os.path.join(SAVE_DIR, "test_predictions.csv")
results_df.to_csv(results_csv, index=False)

# Save report
report_txt = os.path.join(SAVE_DIR, "classification_report.txt")
with open(report_txt, "w") as f:
    f.write(f"Test Accuracy: {acc:.4f}\n\n")
    f.write(report)

print(f"\n🔹 Test Accuracy: {acc:.4f}")
print("\n🔎 Classification Report:")
print(report)
print(f"\n✅ Predictions saved to: {results_csv}")
print(f"✅ Report saved to: {report_txt}")


🔹 Test Accuracy: 0.5365

🔎 Classification Report:
              precision    recall  f1-score   support

    Non-Fake       0.55      0.42      0.47       480
        Fake       0.53      0.65      0.59       480

    accuracy                           0.54       960
   macro avg       0.54      0.54      0.53       960
weighted avg       0.54      0.54      0.53       960


✅ Predictions saved to: /content/drive/MyDrive/MultiBanFake/LSTM_Results/test_predictions.csv
✅ Report saved to: /content/drive/MyDrive/MultiBanFake/LSTM_Results/classification_report.txt
